In [ ]:
import signac
import flow
import numpy as np

from project import MyProject

print(signac.__version__)
print(flow.__version__)

Make sure to run `init.py` at least once to initialize the project!

Then, we get obtain a project handle.

In [ ]:
project = MyProject()
print(project)

Just like on the command line, we can display the project's status:

In [ ]:
project.print_status(detailed=True, parameters=['p'])

We want to compare the volume estimate using the ideal gas law and the measurements from the molecular dynamics simulation.

We implement the `get_volume()` function to extract the volume from the log file.

In [ ]:
def get_volume(job):
    try:
        log = np.genfromtxt(job.fn('dump.log'), names=True)
        N = len(log)
        return log[int(0.5 * N):]['volume'].mean(axis=0)
    except OSError:
        pass

for job in project.find_jobs():
    print(job.statepoint()['p'], get_volume(job))

For a better presentation of the results we need to aggregate all results and sort them by pressure.

*The following code requires matplotlib.*

In [ ]:
# Display plots within the notebook
from matplotlib import pyplot as plt
%matplotlib inline

V = dict()
V_idg = dict()

for job in project.find_jobs():
    V[job.statepoint()['p']] = get_volume(job)
    V_idg[job.statepoint()['p']] = job.document['volume_estimate']
    
p = sorted(V.keys())
V = [V[p_] for p_ in p]
V_idg = [V_idg[p_] for p_ in p]

plt.plot(p, V, label='LJ')
plt.plot(p, V_idg, label='idG')
plt.xlabel(r'pressure [$\epsilon / \sigma^3$]')
plt.ylabel(r'volume [$\sigma^3$]')
plt.legend()